# Training Helper Utilities
To ease the process of training and evaluation multiple models, we have implemented two helper classes `TrainingHelperM` and `TrainingHelperD`, alongside a few helper functions like `get_finetuned_model_name`. These tools help us keep a tidy and safe code.

## `TrainingHelperM`
This class stores metadata regarding the model we are training. Its attributes are the following:
- `HF_model_name`: This is the Hugging Face identifier of the base model we are using.
- `huggingface_prefix`: The prefix string. Name of the developer team.
- `base_model_name`: This is only the name of the model without prefixes.
- `epochs`: Number of training epochs.
- `learning_rate`: Learning rate used to train the model.
- `seq_len`: Max length of the training sequences.
- `batch_size`: Training batch size. This is auto inferred for known model (see below at `__init__()`).
- `gradient_accumulation_steps`: Gradient accumulation steps. Auto-inferred too.

### Methods:
`TrainingHelperM` has a few methods to further help the programmer:
- `from_json(path)`: Loads the model from a JSON file (more below).
- `get_default_model()`: This method returns a Hugging Face AutoModelForSequenceClassification instantiated from the name stored in the helper.
- `get_tokenizer()`: Returns the matching tokenizer to the model.
- `get_tokenizer_function()`: Returns a tokenizer function which can be used by `dataset.map()` for pre-tokenization and other tasks.
- `initialize_from_environment()`: Instantiates the model helper from environmental variables (more below).
- `initialize_from_finetuned_name(name)`: Instantiates the model helper from a finetuned name (more below).
- `to_json(path)`: The helper can save its metadata to a JSON file.

### Initialization of the class
There are four ways to initialize a `TrainingHelperM` class. First off we can do so by calling it's `__init__()` method. Secondly we can use the `initialize_from_environment` class method, to pull the metadata from environment variables. This is extremely helpful for SLURM array jobs, as one can start up a wide range of model trainings from a single launch script. Thirdly we can initialize the model from a JSON file created by the `to_json` method. Lastly we can use the `parse_model_helper_from_finetuned_name` factory function. This one takes a pretrained model name, created by the `get_finetuned_model_name(TrainingHelperD, TrainingHelperM)` function and returns with a `TrainingHelperM` object.

- The `__init__(**kwargs)` method takes the following arguments:
  - huggingface_model_name: (str) Required.
  - epochs: (int) Optional. Defaults to 1.0
  - learning_rate (float) optional. Defaults to 0.001.
  - seq_len (int) Optional. Defaults to 512.
  - batch_size (int) Optional. If not given it's inferred for known models, otherwise an error is thrown.
  - gradient_accumulation_steps (int) Optional. If not given it's inferred for known models, otherwise an error is thrown.
  Since many of the parameters are default or auto inferred those values might not be correct in every case! It is better practice to pass all known information to the `__init__` !

- `initialize_from_environment()`: This is a class method factory function. It looks for the following environmental variables:
  - `MODEL_NAME`: The full Hugging Face name of the model.
  - `LEARNING_RATE`: The learning rate.
  - `LS`: The maximal sequence length for the given training. (The model itself might be able to handle more!)
  - `NUM_TRAIN_EPOCHS`: Number of training epochs.
- `from_json(path)`: This class method loads the class from a JSON file.
- `parse_model_helper_from_finetuned_name(name)`: Not recommended, as  it's unsafe. It only works for known models, as batch size and gradient accumulation steps are not part of the finetuned name, so they have to be auto inferred.

## `TrainingHelperD`
This class serves to store metadata regarding the training and testing dataset used as well as the task name the model was trained for.
- `dataset_path`: Path to the dataset used for training and evaluation. These are Hugging Face datasets so this is a directory containing many splits.
- `eval_split_name`: Name of the evaluation split.
- `seq_len`: Maximum length of sequences in the dataset.
- `task`: Name of the training task.
- `test_dataset_path`: Absolute path to the testing dataset used. Split included!
- `test_split_name`: Name of the test split.
- `train_dataset_path`: Absolute path to the training dataset used. Split included!
- `train_split_name`: Name of the training split.
- `separator`: Short string used to separate metadata in the finetuned name. By default, it is: `___`.
- `val_dataset_path`: Absolute path to the training dataset used. Split included!


### Methods:
This helper is yet to have any convenience methods. It's only two methods are for initialization. See below.

### Initialization
This helper class can be initialized through three methods: passing all variables, to its `__init__()`, calling the `initialize_from_environment` if the environmental variables are present, or loading from a JSON file by the `from_json` method.

- `initialize_from_environment()`: This is a class method factory function. It looks for the following environmental variables:
    - `DATASET_NAME`: Name of the testing dataset.
    - `DATASET_PATH`: Absolute path to the dataset. Without the splits!
    - `LS`: The maximal sequence length for the given training.
    - `TASK`: Name of the training task.

- `from_json(path)` Loads the model from a checkpoint file created by the `to_json` method.

## `get_finetuned_model_name`
This helper function generates the final model name. This is useful to ensure consistency, also making the model names themselves easily parsable. THe function takes two arguments: `get_finetuned_model_name(dataset_helper: TrainingHelperD, model_helper: TrainingHelperM)` and returns with the finetuned name as a string. The name follows this convention: `<DATASET_NAME>___<BASE_MODEL_NAME>___<TASK>___sl_<SEQ_LEN>___ep_<EPOCH>___lr_<LEARNING_RATE>`. The `___` separator string is used by default, but it can be overriden as this is a property of the `TrainingHelperD` class: `TrainingHelperD.separator`.

 # Now that we understand these classes let's try their functions so we gain a deeper insight.
 We will first initialize the classes n multiple ways then extract their properties.


In [ ]:
# This install method is guaranteed to work in google colab, so it is preferred for this example. For more details please check the Readme
#!git clone --single-branch --branch TrainHelper https://github.com/nbrg-ppcu/prokbert.git
#%pip install ./prokbert -q;

In [ ]:
from prokbert.traininghelper_utils import TrainingHelperM, TrainingHelperD, get_finetuned_model_name

In [ ]:
# Let's start by initializing the helpers through their __init__ methods

model_helper = TrainingHelperM(
    huggingface_model_name='neuralbioinfo/prokbert-mini-long')
# Notice that no batch size or gradient accumulation steps is given. These are auto inferred since prokbert-mini-long is a known model
# These batch sizes are calculated assuming 40GB NVIDIA A100-s
del model_helper

# To fully control parameters we can pass in everything through the arguments to init
model_helper = TrainingHelperM(
    huggingface_model_name='neuralbioinfo/prokbert-mini-long',
    epochs=1,
    learning_rate=0.001,
    seq_len=512,
    batch_size=64,
    gradient_accumulation_steps=2
)


In [ ]:
# To initialize the helpers from environmental variables we need to set them up first
import os
os.environ['MODEL_NAME'] = 'neuralbioinfo/prokbert-mini-long'
os.environ['LEARNING_RATE'] = '0.001'
os.environ['LS'] = '256'
os.environ['NUM_TRAIN_EPOCHS'] = '1'
os.environ['TASK'] = 'phage-lifestyle'
os.environ['DATASET_PATH'] = '/tmp/testpath'
os.environ['DATASET_NAME'] = 'testdataset'


In [ ]:
# This will work because prokbert-mini-long is a known model and BS and GAC are auto inferred
del model_helper
model_helper = TrainingHelperM.initialize_from_environment()

In [ ]:
del model_helper
os.environ['MODEL_NAME'] = 'some_developer/some_model'
# This fails because it is not a known model
model_helper = TrainingHelperM.initialize_from_environment()

In [ ]:
# However if we add BS and GAC to the environmental variables the initialization will work
os.environ['BATCH_SIZE'] = '64'
os.environ['GRADIENT_ACCUMULATION_STEPS'] = '4'
model_helper = TrainingHelperM.initialize_from_environment()

In [ ]:
# Lets save the helper so we can try loading it back
model_helper.to_json('model_helper.json')

In [ ]:
# Load the helper from JSON
helper_two = TrainingHelperM.from_json('model_helper.json')
print("The helper loaded back from JSON is equal to the previous", helper_two == model_helper)

In [ ]:
# Let's take a look at the dataset helper now

dataset_helper = TrainingHelperD(
    dataset_path='/path_to_some/dataset',
    dataset_name='testdataset',
    seq_len=512,
    check_dataset_exist=False # This is needed because testdataset does not exist now
) # Note that here we use the default task and separator here: phage and '___'

# We can specify those explicitly. Doing so will change the finetuned name generated
dataset_helper_two = TrainingHelperD(
    dataset_path='/path_to_some/dataset',
    dataset_name='testdataset',
    seq_len=512,
    check_dataset_exist=False,
    separator='***',
    task='phage-lifestyle',
)


In [ ]:
# The dataset helper can be initialized from environment and JSON too
del dataset_helper
del dataset_helper_two
dataset_helper = TrainingHelperD.initialize_from_environment(check_dataset_exist=False)
dataset_helper.to_json('dataset_helper.json')
dataset_helper_two = TrainingHelperD.from_json('dataset_helper.json')

print("The helper loaded back from JSON is equal to the previous", dataset_helper_two == dataset_helper)
del dataset_helper_two

## Now that we know how to initialize and save the classes let's look at their usage
Both classes are decorated with `@dataclass` so they have equivalence and ordering operators (==, <, >, <=, >=). Also since they are dataclasses they can be printed out directly.

In [ ]:
# Let's try printing
print(model_helper)
print(dataset_helper)

### Something look off with those prints?
As you might notice when printed like this the properties have a `_` prefix, and they are different, than described before. This is because these are the hidden internal values that are not supposed to be accessed or changed directly. The public properties of the models are all accessible through getter and setter methods, using the `@property` decorator.

### So what can we access? These public properties
Again the public properties of the helpers are the following:
- `TrainingHelperM`:
    - `HF_model_name`: This is the full Hugging Face identifier of the base model we are using.
    - `huggingface_prefix`: The prefix string. Name of the developer team.
    - `base_model_name`: This is only the name of the model without prefixes.
    - `epochs`: Number of training epochs.
    - `learning_rate`: Learning rate used to train the model.
    - `seq_len`: Max length of the training sequences.
    - `batch_size`: Training batch size. This is auto inferred for known model (see below at `__init__()`).
    - `gradient_accumulation_steps`: Gradient accumulation steps. Auto-inferred too.
- `TrainingHelperD`:
    - `dataset_path`: Path to the dataset used for training and evaluation. These are Hugging Face datasets so this is a directory containing many splits.
    - `eval_split_name`: Name of the evaluation split.
    - `seq_len`: Maximum length of sequences in the dataset.
    - `task`: Name of the training task.
    - `test_dataset_path`: Absolute path to the testing dataset used. Split included!
    - `test_split_name`: Name of the test split.
    - `train_dataset_path`: Absolute path to the training dataset used. Split included!
    - `train_split_name`: Name of the training split.
    - `separator`: Short string used to separate metadata in the finetuned name. By default, it is: `___`.
    - `val_dataset_path`: Absolute path to the training dataset used. Split included!

A few things to note here. The names of the dataset splits are generated from the sequence length used. As a result these are non-modifiable. Trying to do so will raise an error. Consequently, the dataset paths are non-modifiable too!

## Off to the more intriguing functionalities
Here we will showcase the helper functions of these classes. Namely: `TrainingHelperM.get_default_model()`, `TrainingHelperM.get_tokenizer()`, `TrainingHelperM.get_tokenizer_fnction()` and `get_finetuned_model_name`.

In [ ]:
# These are more heavyweight operations it is recommended not to try these without a GPU

model_helper = TrainingHelperM('neuralbioinfo/prokbert-mini-long') # Full default helper for prokbert

model = model_helper.get_default_model() # This will return a Hugging Face AutoModelForClassification
tokenizer = model_helper.get_tokenizer() # This returns the corresponding tokenizer
tokenize_fn = model_helper.get_tokenizer_function() # Tokenizer function to use with dataset.map() for example

In [ ]:
# Let's say we successfully trained out model and would like to save it
finetuned_name = get_finetuned_model_name(dataset_helper=dataset_helper, model_helper=model_helper)

model.save_pretrained(finetuned_name)